<!-- RGB Model -->
<div class="alert" style="background: linear-gradient(to right,rgb(255, 0, 0), rgb(0,255,0),rgb(0, 0, 255)); 
color:rgb(255, 255, 255);">

# **U-NET RGB Colorization Model**
***
### **U-NET RGB Colorization Model.**


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

In [2]:
# Load processed data. Important to run LoadComicData.ipynb first
X_train = np.load("prepared_data/comic_input_grayscale_train.npy")
y_train = np.load("prepared_data/comic_output_color_train.npy")
X_test  = np.load("prepared_data/comic_input_grayscale_test.npy")
y_test  = np.load("prepared_data/comic_output_color_test.npy")

In [ ]:
def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Encoder
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Bottleneck
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    
    # Decoder
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same')(x)
    
    # Output layer (3 channels for RGB)
    outputs = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    model = models.Model(inputs, outputs)
    return model

# Build the model
input_shape = (256, 256, 1)  # Grayscale manga image shape
model = build_unet(input_shape)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit(
    x_train_gray, x_train_color,
    epochs=50,
    batch_size=16)

In [ ]:
model.save("unet_colorizationN2RGB.keras")

In [ ]:
model = load_model("unet_colorizationN2RGB.keras")

In [ ]:
prediction = model.predict(x_test_gray)

In [ ]:
def display_results(original, grayscale, predicted):
    plt.figure(figsize=(20, 5))
    
    plt.subplot(1, 3, 1)
    plt.title('Original')
    plt.imshow(original)

    plt.subplot(1, 3, 2)
    plt.title('Grayscale')
    plt.imshow(grayscale, cmap='gray')
    
    plt.subplot(1, 3, 3)
    plt.title('Predicted')
    plt.imshow(predicted)

    plt.show()

In [ ]:

for i in range(10):
    display_results(x_test_color[i], x_test_gray[i], prediction[i])